In [54]:
from nltk.stem import WordNetLemmatizer
import re
import pandas as pd
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report,ConfusionMatrixDisplay,precision_score, recall_score, f1_score, roc_auc_score,roc_curve

In [55]:

messages=pd.read_csv("SMSSpamcollection.txt",sep='\t',names=['labels','message'])



In [56]:
messages

,labels,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


output varible spam =1 ham=0

In [57]:
y=pd.get_dummies(messages['labels'],dtype=int)
y=y.iloc[:,1].values
y

array([0, 0, 1, ..., 0, 0, 0])

In [58]:

wnl=WordNetLemmatizer()
corpus=[]
for i in range(len(messages)):
    review=re.sub('[^a-zA-z]',' ',messages['message'][i])
    review=review.lower()
    review=review.split()
#    print(review)
    review=[wnl.lemmatize(word,pos='v') for word in review if word not in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)


### Bag of words

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500,ngram_range=(1,2))
x=cv.fit_transform(corpus)
print(x.shape)

(5572, 2500)


In [60]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [61]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((4457, 2500), (1115, 2500), (4457,), (1115,))

In [62]:
models={
    "Logisitic Regression":LogisticRegression(),
    "Random forest classifier":RandomForestClassifier(),
    "Decision tree":DecisionTreeClassifier(),
    "gradient boost":GradientBoostingClassifier(),
    "Support vector machine":SVC(),
    "knearest neighbours":KNeighborsClassifier(),
#    "Naive bias":naive_bayes()
}   
def report(models):

    for i in range(len(list(models))):
        model = list(models.values())[i]
        model.fit(x_train, y_train) # Train model

        # Make predictions
        y_train_pred = model.predict(x_train)
        y_test_pred = model.predict(x_test)

        # Training set performance
        model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
        model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
        model_train_precision = precision_score(y_train, y_train_pred) # Calculate Precision
        model_train_recall = recall_score(y_train, y_train_pred) # Calculate Recall
        model_train_rocauc_score = roc_auc_score(y_train, y_train_pred)
        model_train_confusion_matrix=confusion_matrix(y_train, y_train_pred)


        # Test set performance
        model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
        model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
        model_test_precision = precision_score(y_test, y_test_pred) # Calculate Precision
        model_test_recall = recall_score(y_test, y_test_pred) # Calculate Recall
        model_test_rocauc_score = roc_auc_score(y_test, y_test_pred) #Calculate Roc
        model_test_confusion_matrix=confusion_matrix(y_test, y_test_pred)


        print(list(models.keys())[i])
        
        print('Model performance for Training set')
        print("- Accuracy: {:.4f}".format(model_train_accuracy))
        print('- F1 score: {:.4f}'.format(model_train_f1))
        
        print('- Precision: {:.4f}'.format(model_train_precision))
        print('- Recall: {:.4f}'.format(model_train_recall))
        print('- Roc Auc Score: {:.4f}'.format(model_train_rocauc_score))
        print('- confusion matrix:\n{}'.format(model_train_confusion_matrix))

        
        
        print('----------------------------------')
        
        print('Model performance for Test set')
        print('- Accuracy: {:.4f}'.format(model_test_accuracy))
        print('- F1 score: {:.4f}'.format(model_test_f1))
        print('- Precision: {:.4f}'.format(model_test_precision))
        print('- Recall: {:.4f}'.format(model_test_recall))
        print('- Roc Auc Score: {:.4f}'.format(model_test_rocauc_score))
        print('- confusion matrix:\n{}'.format(model_test_confusion_matrix))
        
        print('='*35)
        print('\n')


In [63]:
report(models)

Logisitic Regression
Model performance for Training set
- Accuracy: 0.9930
- F1 score: 0.9930
- Precision: 1.0000
- Recall: 0.9482
- Roc Auc Score: 0.9741
- confusion matrix:
[[3859    0]
 [  31  567]]
----------------------------------
Model performance for Test set
- Accuracy: 0.9857
- F1 score: 0.9853
- Precision: 0.9926
- Recall: 0.8993
- Roc Auc Score: 0.9491
- confusion matrix:
[[965   1]
 [ 15 134]]


Random forest classifier
Model performance for Training set
- Accuracy: 0.9993
- F1 score: 0.9993
- Precision: 1.0000
- Recall: 0.9950
- Roc Auc Score: 0.9975
- confusion matrix:
[[3859    0]
 [   3  595]]
----------------------------------
Model performance for Test set
- Accuracy: 0.9785
- F1 score: 0.9777
- Precision: 1.0000
- Recall: 0.8389
- Roc Auc Score: 0.9195
- confusion matrix:
[[966   0]
 [ 24 125]]


Decision tree
Model performance for Training set
- Accuracy: 0.9993
- F1 score: 0.9993
- Precision: 1.0000
- Recall: 0.9950
- Roc Auc Score: 0.9975
- confusion matrix:
[[38

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=100,ngram_range=(1,3))
x=cv.fit_transform(corpus)
print(x.shape)

(5572, 100)


In [65]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [66]:
report(models)

Logisitic Regression
Model performance for Training set
- Accuracy: 0.9659
- F1 score: 0.9647
- Precision: 0.9373
- Recall: 0.7993
- Roc Auc Score: 0.8955
- confusion matrix:
[[3827   32]
 [ 120  478]]
----------------------------------
Model performance for Test set
- Accuracy: 0.9641
- F1 score: 0.9628
- Precision: 0.9360
- Recall: 0.7852
- Roc Auc Score: 0.8885
- confusion matrix:
[[958   8]
 [ 32 117]]


Random forest classifier
Model performance for Training set
- Accuracy: 0.9897
- F1 score: 0.9896
- Precision: 0.9726
- Recall: 0.9498
- Roc Auc Score: 0.9728
- confusion matrix:
[[3843   16]
 [  30  568]]
----------------------------------
Model performance for Test set
- Accuracy: 0.9677
- F1 score: 0.9674
- Precision: 0.8951
- Recall: 0.8591
- Roc Auc Score: 0.9218
- confusion matrix:
[[951  15]
 [ 21 128]]


Decision tree
Model performance for Training set
- Accuracy: 0.9897
- F1 score: 0.9896
- Precision: 0.9759
- Recall: 0.9465
- Roc Auc Score: 0.9714
- confusion matrix:
[[38